In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import scipy

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn import model_selection, preprocessing, ensemble
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from scipy import sparse
import xgboost as xgb

import os
import sys
import operator

In [ ]:
df=pd.read_json("../input/train.json")

In [ ]:
df['priceperbed']=(df['price']/df['bedrooms'].clip(lower=1))
df['created']=df['created'].astype(np.datetime64)
df['created_date']=np.array(df.created.values, dtype='datetime64[D]').astype(np.float32)
df['created_day']=np.array(df.created.values, dtype='datetime64[D]').astype(np.float32)%7
df['created_week']=np.array(df.created.values, dtype='datetime64[W]').astype(np.float32)
df['created_month']=np.array(df.created.values, dtype='datetime64[M]').astype(np.float32)
df['created_hour']=np.array(df.created.values, dtype='datetime64[h]').astype(np.float32)%24
df['desc_count']=df.description.apply(lambda x: len(x.split()))
df['features_count']=df.features.apply(lambda x: len(x))
df['photos_count']=df.photos.apply(lambda x: len(x))


In [ ]:
categorical = ["display_address", "building_id", "street_address", "manager_id"]
for f in categorical:
        if df[f].dtype=='object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[f].values) + list(df[f].values))
            df[f] = lbl.transform(list(df[f].values))

In [ ]:
df['features'] = df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
print(df["features"].head())
tfidf = CountVectorizer(stop_words='english', max_features=200)
features_sparse = tfidf.fit_transform(df["features"])

In [ ]:
train_ix, test_ix = train_test_split(range(df.shape[0]), random_state=0)
train_ix, val_ix = train_test_split(train_ix, random_state=0)

In [ ]:
temp = pd.concat([df.iloc[train_ix].manager_id,pd.get_dummies(df.iloc[train_ix].interest_level)], 
                 axis = 1).groupby('manager_id').mean()
temp.columns = ['high_frac','low_frac', 'medium_frac']
temp['count'] = df.iloc[train_ix].groupby('manager_id').count().iloc[:,1]

temp['manager_skill'] = temp['high_frac']*2 + temp['medium_frac']
unranked_managers_ixes = temp['count']<8
ranked_managers_ixes = ~unranked_managers_ixes
mean_values = temp.loc[ranked_managers_ixes, [
    'high_frac','low_frac', 'medium_frac','manager_skill']].mean()
temp.loc[unranked_managers_ixes,[
    'high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values

df = df.merge(temp.reset_index(),how='left', on='manager_id')
new_manager_ixes = df['high_frac'].isnull()
df.loc[new_manager_ixes,['high_frac','low_frac', 'medium_frac','manager_skill'
                            ]] = mean_values.values

In [ ]:
cols=['price', 'bathrooms', 'bedrooms', 'latitude', 'longitude', 'created_hour', 'created_day',
      'created_month', 'created_date', 'desc_count', 'photos_count', 'features_count', 
      'priceperbed', 'manager_id']+categorical

In [ ]:
X = sparse.hstack([df[cols], features_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
y = np.array(df['interest_level'].apply(lambda x: target_num_map[x]))
print(X.shape, y.shape)

In [ ]:
xgtrain = xgb.DMatrix(data=X[train_ix], label=y[train_ix])
xgval = xgb.DMatrix(data=X[val_ix], label=y[val_ix])

In [ ]:
params = {
    #'eta':.15,
    'eta':.15,
    'max_depth':6,
    'min_child_weight':3,
    #'min_child_weight':1,
    'colsample_bytree':.8,
    'subsample':.8,
    #'colsample_bytree':.7,
    #'subsample':.7,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
}

clf = xgb.train(params, xgtrain, 189,#1000, 
                #evals=[(xgtrain, 'train'), (xgval, 'val')], 
                #early_stopping_rounds=15, 
                verbose_eval=25)

In [ ]:
y_pred = clf.predict(xgtrain)
score=log_loss(y[train_ix], y_pred)
print(score)
y_pred = clf.predict(xgval)
score=log_loss(y[val_ix], y_pred)
print(score)